In [1]:
import numpy as np
import copy

In [2]:
test_cases = [
    # [
    #     syst := [
    #         [1, 1, 1, 0],
    #         [1, -1, 0, -1],
    #     ],
    #     b := [2, 3],
    #     res := "задача несовместна"
    # ],
    # [
    #     syst := [
    #         [1, 1, 1],
    #         [2, 2, 2]
    #     ],
    #     b := [0, 0],
    #     res := (
    #         x := [0, 0, 0],
    #         Jb := [0]
    #         )
    # ],
    [
        syst := [
            [-1, 3, 1, 0],
            [4, 3, 0, 1],
            [2, 9, 2, 1],
        ],
        b := [9, 24, 42],
        res:= (
            x := [3, 4, 0, 0],
            Jb := [0, 1]
            ),
    ],
    [
        syst := [
            [-1, 1, 1, 0],
            [1, 1, 0, 1],
            [0, 2, 1, 1],
        ],
        b := [2, 6, 8],
        res := (
            x := [2, 4, 0, 0],
            Jb := [0, 1],
        ),
    ],
]

In [3]:
def InvMatrix(A, x, i):
    IA = np.linalg.inv(A)
    A[:,i] = x[::]
    l = IA.dot(x)
    if (l[i] == 0):
        print("Матрица A необратима")
    ln = l.copy()
    ln[i] = -1
    ls = (-1/l[i]) * ln
    Q = np.eye(len(A))
    Q[:,i] = ls
    return np.matmul(Q, IA)

def Simplex(_syst, _b, _c, _x, _Jb):
    _syst = np.array(copy.deepcopy(_syst))
    _b = np.array(_b[:])
    _c = np.array(_c[:])
    _x = np.array(_x[:])
    _Jb = np.array(_Jb[:])

    n = len(_x)

    j0_new = -1

    while True:
        cb = [] 
        for j in _Jb:
            cb.append(_c[j])

        if (j0_new == -1):
            Ab = np.eye(len(_Jb))
            invAb = np.linalg.inv(Ab)
        else:
            invAb = InvMatrix(Ab, _syst[:,j0], j0_new)

        # вектор потенциалов:
        u = np.dot(cb, invAb)

        # вектор оценок:
        delta = np.matmul(u, _syst) - _c

        optimal = True
        #for i in N:
        for i in range(n):
            if i not in _Jb and delta[i] < 0:
                optimal = False
                break
        
        if optimal:
            F = np.dot(_c, _x)
            print("План отпимальный")
            print(f"F{tuple(list(_x))} = {F}")
            
            return _x, F, _Jb, invAb

        j0 = 0
        for i in range(len(delta)):
            if (delta[i] < 0 and i not in _Jb):
                j0 = i
                break

        z = np.matmul(invAb, _syst[:,j0])
    
        thetas = []  
        for i in range(len(_Jb)):
            thetas.append(_x[_Jb[i]] / z[i] if z[i] > 0 else np.inf)

        theta0 = min(thetas)
        if (theta0 == np.inf):
            print('Целевая функция неограничена сверху')
            return
        j0_new = thetas.index(theta0)

        # если один из индексов базисный, а другой нет -
        # то меняем их местами
        Jb_new = _Jb.copy()
        Jb_new[j0_new] = j0

        # обновляем компоненты текущего базисного множества
        for i in range(len(_x)):
            if (i not in _Jb):
                _x[i] = 0

        for i, j in enumerate(_Jb):
            _x[j] -= theta0 * z[i]
        _x[j0] = theta0

        # базис для следующих итераций
        _Jb[:] = Jb_new

In [4]:
def Simplex_first_phase(_syst, _b):
    n = len(_syst[0])

    _syst = copy.deepcopy(_syst)
    _b = _b[:]

    restrictions_count = len(_syst)

    for i in range(restrictions_count):
        # делаем все свобобдные члены неотрицательными
        if _b[i] < 0:
            _b[i] *= -1
            _syst[i] = [-j for j in _syst[i]]

        # вводим исскуственные переменные
        _syst[i] += [0] * restrictions_count
        _syst[i][-restrictions_count + i] = 1

    # чтобы найти начальный базисный план,
    # значения неискуственных переменных положим равными 0
    _x = [0] * n + _b

    _Jb = []
    for i in range(-restrictions_count, 0):
        _Jb.append(i + len(_x))

    _c = [0] * (n + restrictions_count)

    for j in _Jb:
        _c[j] = -1

    _x, F, _Jb, invAb = Simplex(_syst, _b, _c, _x, _Jb)

    for i in range(len(_x) - restrictions_count, len(_x)):
        if (_x[i] != 0):
            print("Задача несовместна")
            return

    print("Задача совместна")
    
    _x = _x[:n]
    print(f"x\' = {tuple(_x)}")

    # находим множество базисных индексов
    while True:
        _syst = np.array(_syst)
        # вычисляем l(j) для каждого небазисного индекса
        have_excess = True
        for j in range(n):
            if (j in _Jb):
                continue

            lj = invAb.dot(_syst[:,j])

            for i, j in enumerate(_Jb):
                # если индекс небазисной переменной
                if j > n:
                    if i < len(lj) and lj[i] != 0:
                        _Jb[i] = j
                        have_excess = False
                        break

        _syst = list(_syst)
        for i in range(len(_syst)):
            _syst[i] = list(_syst[i])
        _Jb = list(_Jb)

        # если есть избыточные ограничения
        if (have_excess):
            for j in _Jb:
                if j > n:
                    # удаляем избыточное ограничение
                    excess = j - n
                    _syst = _syst[:excess] + _syst[excess + 1:]

                    # удаляем из базисного множества
                    _Jb.remove(j)

                    # удаляем переменную
                    for i in range(len(_syst)):
                        _syst[i] = _syst[i][:j] + _syst[i][j + 1:]

                    break

        # проверяем, если остались искусственные базисы
        has_artificial_basis = False
        for j in _Jb:
            if j > n:
                has_artificial_basis = True

        if (has_artificial_basis):
            continue

        print(f"Базисный план: {set(_Jb)}")
        break


In [5]:
for test_case in test_cases:
    *data, res = test_case
    Simplex_first_phase(*data)
    
    if type(res) != str:
        res = f"x' = {tuple(res[0])}, Jb = {set(res[1])}"
    print("-" * 10)
    print(f'Правильный ответ:\n{res}\n\n')

План отпимальный
F(3, 4, 0, 0, 0, 0, 0) = 0
Задача совместна
x' = (3, 4, 0, 0)
Базисный план: {0, 1}
----------
Правильный ответ:
x' = (3, 4, 0, 0), Jb = {0, 1}


План отпимальный
F(2, 4, 0, 0, 0, 0, 0) = 0
Задача совместна
x' = (2, 4, 0, 0)
Базисный план: {0, 1}
----------
Правильный ответ:
x' = (2, 4, 0, 0), Jb = {0, 1}


